In [1]:
# Load packages
library(readxl)
library(stringr)
library(rentrez)
library(dplyr)
library(tidyr)
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# Adjust PATHs

## Location in directory
wd <- getwd()
subdir <- "/bin" # should be 'scripts' if location is ./scripts/thisfile
if (endsWith(wd, subdir)) {
    wd <- str_remove(wd, subdir)
    setwd(wd)
}
cat('Directory: ', getwd())

## Conda environment
conda_env <- paste0('/home/',Sys.getenv('USER'),'/miniconda3/envs/arabinosylation-anti-crispr/bin')
PATH <- Sys.getenv('PATH')
if (!str_detect(PATH, conda_env)) {
    Sys.setenv('PATH' = paste(conda_env, PATH, sep = ':'))
}
Sys.getenv('PATH')

Directory:  /projects/health_sciences/bms/microbiology/fineranlab/arabinosylation-anti-CRISPR

[1] "/home/dieol22p/miniconda3/envs/arabinosylation-anti-crispr/bin:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"

In [3]:
# Load internal functions
source('bin/utils.R')

In [4]:
# Variables
in_url <- 'https://millardlab-inphared.s3.climb.ac.uk/1Jan2025_data.tsv.gz'
out_path <- 'data/millard_list/'
dir.create(out_path)

fn <- list(
    tsv = 'data/millard_list/list.tsv',
    accession = 'data/millard_list/accession.txt'
)
fn

Warning message in dir.create(out_path):
“'data/millard_list' already exists”


$tsv
[1] "data/millard_list/list.tsv"

$accession
[1] "data/millard_list/accession.txt"

In [5]:
# Download data
download.file(in_url, fn$tsv)

In [6]:
# Read data
data <- readr::read_tsv(fn$tsv)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 32043 Columns: 27
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (18): Accession, Description, Classification, Molecule, Modification Dat...
dbl  (7): Genome Length (bp), molGC (%), Number CDS, Positive Strand (%), Ne...
lgl  (2): Jumbophage, Low Coding Capacity Warning

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
# Summary

## View
head(data, 2)

dim(data)
length(unique(data$Accession))
any(is.na(data$Accession))
length(unique(data$Description))
any(is.na(data$Description))

Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,Positive Strand (%),⋯,Sub-family,Family,Order,Class,Phylum,Kingdom,Realm,Baltimore Group,Genbank Division,Isolation Host (beware inconsistent and nonsense values)
<chr>,<chr>,<chr>,<dbl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AY319521,Salmonella phage SopEPhi,Salmonella phage SopEPhi Felsduovirus SopEphi Felsduovirus Peduoviridae Caudoviricetes Uroviricota Heunggongvirae Duplodnaviria Viruses,35155,FALSE,51.321,DNA,22-MAR-2024,45,82.22222,⋯,Unclassified,Peduoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,SYN,Salmonella enterica subsp. enterica serovar Typhimurium
MW175890,Dompiswa phage TSP7_1,Dompiswa phage TSP7_1 Phapecoctavirus TSP7 Phapecoctavirus Stephanstirmvirinae Caudoviricetes Uroviricota Heunggongvirae Duplodnaviria Viruses,150892,FALSE,39.115,DNA,10-JAN-2024,272,49.63235,⋯,Stephanstirmvirinae,Unclassified,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Unspecified


[1] 32043    27

[1] 32043

[1] FALSE

[1] 23233

[1] FALSE

In [25]:
## Create groups of IDs
n_groups <- ceiling(nrow(data) / 100)
data$batch <- factor(sample(1:n_groups, nrow(data), replace=TRUE))

entrez_ids <- list()
acc_ids <- list()
for (i in levels(data$batch)) {
    ind <- which(data$batch == i)
    entrez_ids[[i]] <- entrez_search(db = 'nuccore', term = paste(data$Accession[ind], collapse = ' '))$ids
    if (length(entrez_ids[[i]]) < 1) {next}
    acc_ids[[i]] <- str_split(entrez_fetch('nuccore', entrez_ids[[i]], rettype = 'acc'),'\\n')[[1]]
}

In [31]:
length(unlist(acc_ids))
# Well fuck me this is shit

[1] 6321

In [38]:
x <- str_split(unlist(acc_ids),'\\.',simplify=TRUE)[,1]
x <- x[which(x != '')]
table(data$Accession %in% x)
# Even worse. How the fuck can they not just provide correct IDs?


FALSE  TRUE 
26023  6020 

In [41]:
# Export accession numbers
# writeLines(data$Accession, fn$accession)

In [42]:
# Download genomes
# download_ncbi_genomes(inputfile = fn$accession)

datasets download virus genome accession --inputfile data/millard_list/accession.txt --filename data/millard_list/ncbi_dataset.zip --include genome,cds,protein,annotation,biosample > data/millard_list/download.log 2>&1 



Warning message in system(call_datasets, intern = TRUE):
“running command 'datasets download virus genome accession --inputfile data/millard_list/accession.txt --filename data/millard_list/ncbi_dataset.zip --include genome,cds,protein,annotation,biosample > data/millard_list/download.log 2>&1' had status 1”


Unzip



Warning message in unzip(zipfile, exdir = PATH):
“error 1 in extracting from zip file”
Warning message in file(file, "rt"):
“cannot open file 'data/millard_list/md5sum.txt': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [43]:
readLines('data/millard_list/download.log')

[1] "Error: No valid accessions were passed"                                                          
[2] ""                                                                                                
[3] "Use datasets download virus genome accession <command> --help for detailed help about a command."
[4] ""

In [39]:
sessionInfo()

R version 4.4.2 (2024-10-31)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 22.04.4 LTS

Matrix products: default
BLAS/LAPACK: /home/dieol22p/miniconda3/envs/arabinosylation-anti-crispr/lib/libopenblasp-r0.3.28.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggplot2_3.5.1 tidyr_1.3.1   dplyr_1.1.4   rentrez_1.2.3 stringr_1.5.1
[6] readxl_1.4.3 

loaded via a namespace (and not attached):
 [1] bit_4.5.0.1       gtable_0.3.6      jsonlite_1.8.9    compiler_4.4.2   
 [5] crayon_1.5.3      tidyselect_1.2.1  IRdisplay_1.1     pa